In [1]:
import pandas as pd, numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer   
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
from scipy import sparse
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.feature_selection import SelectFromModel

In [2]:
column = "word_seg"
train = pd.read_csv('../input/new_data/train_set.csv')
test = pd.read_csv('../input/new_data/test_set.csv')
test_id = test["id"].copy()
y_all = (pd.Series(train['class'])-1).astype(int)

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [3]:
# trn_term_doc = sparse.load_npz('../feature/text_tifi/trn_term_doc.npz')
# trn_term_word = sparse.load_npz('../feature/text_tifi/trn_term_word.npz')
# trn_term_word_three = sparse.load_npz('../feature/text_tifi/trn_term_word_three.npz')
# train_feaure =  hstack([trn_term_doc,trn_term_word,trn_term_word_three]).tocsr()
# del trn_term_doc,trn_term_word,trn_term_word_three
# import gc
# gc.collect()
# trn_term_word_four = sparse.load_npz('../feature/text_tifi/trn_term_word_four.npz')
# # trn_term_word_five = sparse.load_npz('../feature/text_tifi/trn_term_word_five.npz')
# trn_char = sparse.load_npz('../feature/text_tifi/trn_char.npz')
# docLen  =pd.DataFrame( np.array(train[column].map(lambda x : len(x.split(" ")))/39759).reshape(-1,1))
# doclen_word = pd.DataFrame( np.array(train['article'].map(lambda x : len(x.split(" ")))/train[column].map(lambda x : len(x.split(" ")))).reshape(-1,1))
# docLen_test  =pd.DataFrame( np.array(test[column].map(lambda x : len(x.split(" ")))/39759).reshape(-1,1))
# doclen_word_test = pd.DataFrame( np.array(test['article'].map(lambda x : len(x.split(" ")))/test[column].map(lambda x : len(x.split(" ")))).reshape(-1,1))

In [4]:
import gc
del train, test
gc.collect()

14

In [5]:
from keras.utils import np_utils
one_y_train=np_utils.to_categorical(y_all,num_classes=20)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
# train_feaure =  hstack([train_feaure, trn_term_word_four,trn_char,doclen_word]).tocsr()
# del trn_term_word_four,trn_char,doclen_word
# gc.collect()

In [7]:
# test_term_doc = sparse.load_npz('../feature/text_tifi/test_term_doc.npz')
# test_term_word = sparse.load_npz('../feature/text_tifi/test_term_word.npz')
# test_term_word_three = sparse.load_npz('../feature/text_tifi/test_term_word_three.npz')
# test_term_word_four = sparse.load_npz('../feature/text_tifi/test_term_word_four.npz')
# # test_term_word_five = sparse.load_npz('../feature/text_tifi/test_term_word_five.npz')
# test_char = sparse.load_npz('../feature/text_tifi/test_char.npz')
# test_festure =  hstack([test_term_doc,test_term_word,test_term_word_three,test_term_word_four,test_char,doclen_word_test]).tocsr()
# del test_term_doc,test_term_word,test_term_word_three,test_term_word_four,test_char,doclen_word_test
# gc.collect()

In [9]:
import pickle
svd_dir = '../feature/svd_5.pkl'
tifi_dir = '../feature/tifi_min4.pkl'
train_svd, test_svd= pickle.load(open(svd_dir,'rb'))
trn_term_doc, test_term_doc= pickle.load(open(tifi_dir,'rb'))
train_feaure = hstack((trn_term_doc, train_svd)).tocsr()
test_festure = hstack((test_term_doc, test_svd)).tocsr()

In [10]:
# X_train, X_valid, Y_train, Y_valid = train_test_split(train_feaure, y_all, test_size=0.15, random_state=42)

In [11]:
name = ["class_prob_%s"%i for i in range(1,20)]
one_y_train=np_utils.to_categorical(y_all,num_classes=20)
# one_y_val=np_utils.to_categorical(Y_valid,num_classes=20)
test_prob = pd.DataFrame([],columns=["class_prob_%s"%i for i in range(1,20)])
val_prob = pd.DataFrame([],columns=["class_prob_%s"%i for i in range(1,20)])
train_prob = pd.DataFrame([],columns=["class_prob_%s"%i for i in range(1,20)])

In [12]:
def ridge_f1_score(y_hat, data):
    y_true = data
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return f1_score(y_true, y_hat, average='macro')

In [13]:
from sklearn.linear_model import Ridge
from tqdm import tqdm
from sklearn.model_selection import KFold

def kf_train(fold_cnt=3,rnd=1):
    now_nfold=0
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((102277,19)),np.zeros((102277,19))
    for train_index, test_index in kf.split(train_feaure):
        now_nfold+=1
        print ("now is {} fold".format(now_nfold))
        curr_x1, hold_out_x1 = train_feaure[train_index], train_feaure[test_index]
        curr_y, hold_out_y = one_y_train[train_index], one_y_train[test_index]
        for i in tqdm(range(19)):
            model = Ridge(solver="sag", fit_intercept=True, random_state=205,alpha=0.1)
            model.fit(curr_x1, curr_y[:,i])
            test_pred[:,i] += model.predict(test_festure)
            ypred = model.predict(hold_out_x1)
            print ("{}line f1 score is {}".format(i, ridge_f1_score(ypred, hold_out_y[:,i])))
            train_pred[test_index,i] = ypred
    test_pred = test_pred / fold_cnt
    return train_pred, test_pred

In [14]:
train_pred,test_pred = kf_train(fold_cnt=10,rnd=4)

now is 1 fold


  5%|▌         | 1/19 [02:43<48:57, 163.19s/it]

0line f1 score is 0.744043470433427


 11%|█         | 2/19 [04:34<38:55, 137.36s/it]

1line f1 score is 0.860592503507717


...

16line f1 score is 0.8539288544479637


 95%|█████████▍| 18/19 [43:55<02:26, 146.42s/it]

17line f1 score is 0.6131070839066182


100%|██████████| 19/19 [46:30<00:00, 146.88s/it]

18line f1 score is 0.7544342888280814


In [ ]:
# val_result = np.argmax(train_prob[name].values,axis=1)
# print ('score is {}'.format(f1_score(y_all, val_result, average='macro')))

In [ ]:
# test_prob["id"]=list(test_id)
# test_prob.to_csv('../pro/prob_ridge_baseline4.csv',index=None)

In [15]:
import pickle
with open('../feature/stacking_pkl_file/10_fold_ridege.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)